In [ ]:
import os, boto3

In [ ]:
# Retrieve the AWS credentials and region from environment variables
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")

print(f"AWS Access Key ID: {aws_access_key_id}")
print(f"AWS Secret Access Key: {aws_secret_access_key}")
print(f"Region: {region_name}")

In [ ]:
# create VPC
ec2_client = boto3.client(
    "ec2",
    region_name=region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)
vpc_name = "vpc-creation-n-interaction-study"

# check if vpc exists

vpc_exists = False
vpc_id = None

response = ec2_client.describe_vpcs(
    Filters=[{"Name": "tag:Name", "Values": [vpc_name]}]
)

vpcs = response.get("Vpcs", [])

vpc_cidr = "10.0.0.0/16"

if vpcs:
    vpc_id = vpcs[0]["VpcId"]
    print(f"VPC with name '{vpc_name}' and ID '{vpc_id}' already exists.")
else:
    vpc_response = ec2_client.create_vpc(CidrBlock=vpc_cidr)
    vpc_id = vpc_response["Vpc"]["VpcId"]

# check if vpc was created if not created retry and if it has the correct tags
if vpc_id:
    ec2_client.create_tags(
        Resources=[vpc_id],
        Tags=[{"Key": "Name", "Value": vpc_name}],
    )
    print(f"VPC with name '{vpc_name}' and ID '{vpc_id}' has been created and tagged.")
else:
    print("Failed to create VPC.")


In [ ]:
# create new internet gateway
igw_name = "igw-creation-n-interaction-study"

# check if internet gateway exists
response = ec2_client.describe_internet_gateways(
    Filters=[{"Name": "tag:Name", "Values": [igw_name]}]
)

igws = response.get("InternetGateways", [])

if igws:
    igw_id = igws[0]["InternetGatewayId"]
    print(f"Internet Gateway with name '{igw_name}' and ID '{igw_id}' already exists.")
else:
    igw_response = ec2_client.create_internet_gateway()
    igw_id = igw_response["InternetGateway"]["InternetGatewayId"]
    ec2_client.create_tags(
        Resources=[igw_id],
        Tags=[{"Key": "Name", "Value": igw_name}],
    )
    print(
        f"Internet Gateway with name '{igw_name}' and ID '{igw_id}' has been created and tagged."
    )
    ec2_client.attach_internet_gateway(VpcId=vpc_id, InternetGatewayId=igw_id)

In [ ]:
route_table_name = "route-table-creation-n-interaction-study"

# check if route table exists
response = ec2_client.describe_route_tables(
    Filters=[{"Name": "tag:Name", "Values": [route_table_name]}]
)

route_tables = response.get("RouteTables", [])

if route_tables:
    route_table_id = route_tables[0]["RouteTableId"]
    print(
        f"Route Table with name '{route_table_name}' and ID '{route_table_id}' already exists."
    )
else:
    route_table_response = ec2_client.create_route_table(VpcId=vpc_id)
    route_table_id = route_table_response["RouteTable"]["RouteTableId"]
    route = ec2_client.create_route(
        RouteTableId=route_table_id,
        DestinationCidrBlock="0.0.0.0/0",
        GatewayId=igw_id,
    )

    ec2_client.create_tags(
        Resources=[route_table_id],
        Tags=[{"Key": "Name", "Value": route_table_name}],
    )

    print(f"Route Table with ID '{route_table_id}' has been created.")

In [ ]:
# get list of availability zones
response = ec2_client.describe_availability_zones()
availability_zones = [zone["ZoneName"] for zone in response["AvailabilityZones"]]
print(f"Availability Zones: {availability_zones}")

# for each az check if subnet exists
# if not create a public subnet

for index,az in enumerate(availability_zones):
    subnet_cidr = f"10.0.{index}.0/24"
    subnet_name = f"public-subnet-{index + 1}"
    response_subnets = ec2_client.describe_subnets(
        Filters=[
            {"Name": "tag:Name", "Values": [subnet_name]},
            {"Name": "vpc-id", "Values": [vpc_id]},
        ]
    )
    response_cidr = ec2_client.describe_subnets(
        Filters=[
            {"Name": "cidrBlock", "Values": [subnet_cidr]},
            {"Name": "vpc-id", "Values": [vpc_id]},
        ]
    )
    subnets = response_subnets.get("Subnets", []) + response_cidr.get("Subnets", [])
    if subnets:
        subnet_id = subnets[0]["SubnetId"]
        print(f"Public Subnet with name '{subnet_name}' and ID '{subnet_id}' already exists in availability zone {az}.")
    else:
        subnet_response = ec2_client.create_subnet(
            VpcId=vpc_id, CidrBlock=subnet_cidr, AvailabilityZone=az
        )
        subnet_id = subnet_response["Subnet"]["SubnetId"]
        ec2_client.create_tags(
            Resources=[subnet_id],
            Tags=[{"Key": "Name", "Value": subnet_name}],
        )
        print(f"Public Subnet with name '{subnet_name}' and ID '{subnet_id}' has been created in availability zone {az}.")


In [ ]:
# steps to delete the vpc and its resources
# 1. delete all the resources attached to the vpc
# 2. delete the vpc

# delete all the resources attached to the vpc
# delete subnets

# if subnets exist delete
response = ec2_client.describe_subnets(Filters=[{"Name": "vpc-id", "Values": [vpc_id]}])
subnets = response.get("Subnets", [])

for subnet in subnets:
    subnet_id = subnet["SubnetId"]
    ec2_client.delete_subnet(SubnetId=subnet_id)
    print(f"Subnet with ID '{subnet_id}' has been deleted.")

# if vpc exists delete
response = ec2_client.describe_vpcs(
    Filters=[{"Name": "tag:Name", "Values": [vpc_name]}]
)

vpcs = response.get("Vpcs", [])

if vpcs:
    vpc_id = vpcs[0]["VpcId"]
    ec2_client.delete_vpc(VpcId=vpc_id)
    print(f"VPC with ID '{vpc_id}' has been deleted.")
else:
    print("VPC does not exist.")

# delete route table
response = ec2_client.describe_route_tables(Filters=[{"Name": "vpc-id", "Values": [vpc_id]}])
route_tables = response.get("RouteTables", [])
for route_table in route_tables:
    route_table_id = route_table["RouteTableId"]
    ec2_client.delete_route_table(RouteTableId=route_table_id)
    print(f"Route Table with ID '{route_table_id}' has been deleted.")
    
# detach and delete internet gateway
response = ec2_client.describe_internet_gateways(Filters=[{"Name": "attachment.vpc-id", "Values": [vpc_id]}])
igws = response.get("InternetGateways", [])
for igw in igws:
    igw_id = igw["InternetGatewayId"]
    ec2_client.detach_internet_gateway(VpcId=vpc_id, InternetGatewayId=igw_id)
    ec2_client.delete_internet_gateway(InternetGatewayId=igw_id)
    print(f"Internet Gateway with ID '{igw_id}' has been detached and deleted.")
    
